In [1]:
import pickle
import pandas as pd
import fasttext
import numpy as np
import os

# CSV dosyasını yükleme
print("CSV dosyası yükleniyor...")
df = pd.read_csv("data_all_news.csv")
print(f"CSV dosyası başarıyla yüklendi. Toplam satır sayısı: {len(df)}")

# FastText modelini yükleme
print("FastText modeli yükleniyor...")
fasttext.FastText.eprint = lambda x: None
model = fasttext.load_model('models/cc.tr.300.bin')  # Türkçe FastText modeli
print("FastText modeli başarıyla yüklendi.")

# Kelime vektörlerini önceden hesaplama
print("Modelden kelime vektörleri çekiliyor...")
word_vectors = {word: model.get_word_vector(word) for word in model.get_words()}
print(f"Toplam kelime vektörü sayısı: {len(word_vectors)}")

# Haber başlıklarını vektörleme
def fasttext_vectorize(text, word_vectors, dim=300):
    words = text.split()
    vectors = [word_vectors[word] for word in words if word in word_vectors]
    return np.mean(vectors, axis=0) if vectors else np.zeros(dim)

# Parça başına işleme ve kaydetme
batch_size = 100
output_file = "fasttext_vectors.pkl"

# Eğer çıktı dosyası varsa kaldığımız yerden devam etmek için yükleme
if os.path.exists(output_file):
    print("Mevcut çıktı dosyası bulunuyor. İşlem kaldığı yerden devam edecek...")
    with open(output_file, "rb") as file:
        existing_data = pickle.load(file)
        X, y = existing_data
        X, y = list(X), list(y)  # Listeye çevirip ekleme yapacağız
    processed_count = len(X)
    print(f"{processed_count} başlık daha önce işlenmiş.")
else:
    print("Yeni işlem başlatılıyor...")
    X, y = [], []
    processed_count = 0

# İşleme devam et
print("Başlıklar işleniyor...")
for i in range(processed_count, len(df), batch_size):
    batch = df.iloc[i:i+batch_size]
    print(f"Batch işlemi başlıyor: {i} - {i + len(batch)}")
    batch_vectors = [fasttext_vectorize(title, word_vectors) for title in batch['Title']]
    X.extend(batch_vectors)
    y.extend(batch['Label'])
    print(f"Batch tamamlandı: {i} - {i + len(batch)}. Şimdi dosyaya kaydediliyor...")

    # Her batch sonrası dosyayı güncelleme
    with open(output_file, "wb") as file:
        pickle.dump((X, y), file)
    print(f"Batch dosyaya kaydedildi: {i} - {i + len(batch)}")

print("Tüm veriler başarıyla işlendi ve kaydedildi.")


: 

In [ ]:
from sklearn.model_selection import train_test_split

# PKL dosyasını yükleme
with open("fasttext_vectors.pkl", "rb") as file:
    X, y = pickle.load(file)

# Eğitim ve test veri setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Eğitim verisi boyutu:", X_train.shape)
print("Test verisi boyutu:", X_test.shape)
